In [1]:
import os
os.chdir('../')

import transtab

# set random seed
transtab.random_seed(42)

In [3]:
trainset

[(     own_telephone  foreign_worker  duration  credit_amount  \
  636              0               1  0.294118       0.061957   
  182              0               1  0.250000       0.076868   
  736              0               1  0.294118       0.622318   
  922              0               1  0.073529       0.061406   
  511              1               1  0.470588       0.244085   
  ..             ...             ...       ...            ...   
  845              1               1  0.250000       0.205018   
  492              0               1  0.029412       0.054308   
  849              0               1  0.117647       0.025256   
  297              0               0  0.088235       0.057060   
  98               0               1  0.470588       0.114834   
  
       installment_commitment  residence_since       age  existing_credits  \
  636                1.000000         0.000000  0.160714          0.000000   
  182                1.000000         0.333333  0.375000     

In [2]:
# load a dataset and start vanilla supervised training
allset, trainset, valset, testset, cat_cols, num_cols, bin_cols = transtab.load_data(['credit-g', 'credit-approval'])

# build transtab classifier model
model = transtab.build_classifier(cat_cols, num_cols, bin_cols)

# start training
training_arguments = {
    'num_epoch':50,
    'eval_metric':'val_loss',
    'eval_less_is_better':True,
    'output_dir':'./checkpoint',
    'batch_size':128,
    'lr':1e-4,
    'weight_decay':1e-4,
    }
transtab.train(model, trainset[0], valset[0], **training_arguments)

# save model
model.save('./ckpt/pretrained')

########################################


/home/txl4827/miniconda3/envs/transtab/lib/python3.9/site-packages/openml/datasets/functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


openml data index: 31
load data from credit-g
# data: 1000, # feat: 20, # cate: 11,  # bin: 2, # numerical: 7, pos rate: 0.70
########################################


/home/txl4827/projects/transtab/transtab/dataset.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X[bin_cols] = X[bin_cols].astype(str).applymap(lambda x: 1 if x.lower() in ['yes','true','1','t'] else 0).values
/home/txl4827/miniconda3/envs/transtab/lib/python3.9/site-packages/openml/datasets/functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


openml data index: 29
load data from credit-approval
# data: 690, # feat: 15, # cate: 9,  # bin: 0, # numerical: 6, pos rate: 0.56


...

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

epoch: 0, test val_loss: 0.574102
epoch: 0, train loss: 3.9759, lr: 0.000100, spent: 2.2 secs
epoch: 1, test val_loss: 0.565162
epoch: 1, train loss: 3.7812, lr: 0.000100, spent: 3.1 secs
epoch: 2, test val_loss: 0.576745
EarlyStopping counter: 1 out of 5
epoch: 2, train loss: 3.6560, lr: 0.000100, spent: 3.9 secs
epoch: 3, test val_loss: 0.566665
EarlyStopping counter: 2 out of 5
epoch: 3, train loss: 3.6539, lr: 0.000100, spent: 4.7 secs
epoch: 4, test val_loss: 0.548929
epoch: 4, train loss: 3.6118, lr: 0.000100, spent: 5.7 secs
epoch: 5, test val_loss: 0.545801
epoch: 5, train loss: 3.5634, lr: 0.000100, spent: 6.6 secs
epoch: 6, test val_loss: 0.545121
epoch: 6, train loss: 3.5035, lr: 0.000100, spent: 7.5 secs
epoch: 7, test val_loss: 0.529129
epoch: 7, train loss: 3.4372, lr: 0.000100, spent: 8.4 secs
epoch: 8, test val_loss: 0.525150
epoch: 8, train loss: 3.3768, lr: 0.000100, spent: 9.2 secs
epoch: 9, test val_loss: 0.518059
epoch: 9, train loss: 3.3204, lr: 0.000100, spent: 1

2024-01-07 14:48:22.130 | INFO     | transtab.trainer:train:136 - load best at last from ./checkpoint
2024-01-07 14:48:22.154 | INFO     | transtab.trainer:save_model:247 - saving model checkpoint to ./checkpoint


epoch: 38, test val_loss: 0.504166
EarlyStopping counter: 5 out of 5
early stopped


2024-01-07 14:48:22.334 | INFO     | transtab.trainer:train:141 - training complete, cost 33.3 secs.


In [3]:
# now let's use another data and try to leverage the pretrained model for finetuning
# here we have loaded the required data `credit-approval` before, no need to load again.

# load the pretrained model
model.load('./ckpt/pretrained')

# update model's categorical/numerical/binary column dict
# need to specify the number of classes if the new dataset has different # of classes from the 
# pretrained one.
model.update({'cat':cat_cols,'num':num_cols,'bin':bin_cols, 'num_class':2})

2023-11-09 22:21:25.638 | INFO     | transtab.modeling_transtab:load:774 - missing keys: []
2023-11-09 22:21:25.640 | INFO     | transtab.modeling_transtab:load:775 - unexpected keys: []
2023-11-09 22:21:25.641 | INFO     | transtab.modeling_transtab:load:776 - load model from ./ckpt/pretrained
2023-11-09 22:21:25.665 | INFO     | transtab.modeling_transtab:load:222 - load feature extractor from ./ckpt/pretrained/extractor/extractor.json


In [4]:
# start training
training_arguments = {
    'num_epoch':50,
    'eval_metric':'auc',
    'eval_less_is_better':False,
    'output_dir':'./checkpoint',
    'batch_size':128,
    'lr':2e-4,
    }

transtab.train(model, trainset[1], valset[1], **training_arguments)


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

epoch: 0, test auc: 0.877056
epoch: 0, train loss: 2.7236, lr: 0.000200, spent: 0.5 secs
epoch: 1, test auc: 0.898701
epoch: 1, train loss: 2.3199, lr: 0.000200, spent: 0.8 secs
epoch: 2, test auc: 0.901299
epoch: 2, train loss: 1.9922, lr: 0.000200, spent: 1.2 secs
epoch: 3, test auc: 0.902165
epoch: 3, train loss: 1.9006, lr: 0.000200, spent: 1.5 secs
epoch: 4, test auc: 0.903030
epoch: 4, train loss: 1.9025, lr: 0.000200, spent: 2.0 secs
epoch: 5, test auc: 0.903896
epoch: 5, train loss: 1.9316, lr: 0.000200, spent: 2.4 secs
epoch: 6, test auc: 0.904762
epoch: 6, train loss: 1.8867, lr: 0.000200, spent: 2.7 secs
epoch: 7, test auc: 0.907359
epoch: 7, train loss: 1.8282, lr: 0.000200, spent: 3.1 secs
epoch: 8, test auc: 0.905628
EarlyStopping counter: 1 out of 5
epoch: 8, train loss: 1.8394, lr: 0.000200, spent: 3.4 secs
epoch: 9, test auc: 0.904762
EarlyStopping counter: 2 out of 5
epoch: 9, train loss: 1.8135, lr: 0.000200, spent: 3.8 secs
epoch: 10, test auc: 0.905628
EarlyStoppin

2023-11-09 22:21:30.570 | INFO     | transtab.trainer:train:136 - load best at last from ./checkpoint
2023-11-09 22:21:30.584 | INFO     | transtab.trainer:save_model:247 - saving model checkpoint to ./checkpoint
2023-11-09 22:21:30.763 | INFO     | transtab.trainer:train:141 - training complete, cost 5.0 secs.


epoch: 12, test auc: 0.906494
EarlyStopping counter: 5 out of 5
early stopped


In [6]:
# evaluation
x_test, y_test = testset[1]
ypred = transtab.predict(model, x_test, y_test)
transtab.evaluate(ypred, y_test, metric='auc')

from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, ypred))

<class 'pandas.core.series.Series'>
591    0
280    1
688    1
345    1
433    1
      ..
270    0
640    1
503    0
472    1
56     0
Length: 138, dtype: int64
0.8516074089844582


In [8]:
transtab.predict(model, x_test, y_test)

<class 'pandas.core.series.Series'>
591    0
280    1
688    1
345    1
433    1
      ..
270    0
640    1
503    0
472    1
56     0
Length: 138, dtype: int64


array([0.10238233, 0.9414178 , 0.9487131 , 0.94316643, 0.9472686 ,
       0.6237414 , 0.94226485, 0.8916272 , 0.9362268 , 0.41641104,
       0.12146674, 0.91955036, 0.12717126, 0.83559066, 0.28000513,
       0.41722918, 0.4239978 , 0.35411915, 0.29504532, 0.9425222 ,
       0.40128878, 0.311141  , 0.71771014, 0.9304242 , 0.969557  ,
       0.9414158 , 0.21258257, 0.4463454 , 0.8610884 , 0.43519256,
       0.7737714 , 0.7846596 , 0.11704271, 0.4133278 , 0.9501856 ,
       0.9762373 , 0.96964204, 0.4014582 , 0.1502246 , 0.3293824 ,
       0.7861262 , 0.969995  , 0.3781279 , 0.21768375, 0.50117564,
       0.78088725, 0.9404731 , 0.9657596 , 0.886171  , 0.9669822 ,
       0.82203037, 0.3309794 , 0.9524497 , 0.4305382 , 0.7775538 ,
       0.14723192, 0.90881115, 0.4096424 , 0.12045365, 0.3447752 ,
       0.6539435 , 0.86244327, 0.92723995, 0.38572398, 0.47441232,
       0.90979624, 0.9692085 , 0.9273723 , 0.946693  , 0.9141363 ,
       0.9170017 , 0.9060644 , 0.82174426, 0.9603108 , 0.69980